# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [19]:
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [20]:
sales_connection = sqlite3.connect('go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [21]:
tables_to_read = ['product', 'product_type', 'product_line', 'sales_staff', 'sales_branch', 
                  'retailer_site', 'country', 'order_header', 'order_details',
                  'returned_item', 'return_reason']

dataframes = {}

for table in tables_to_read:
    query = f"SELECT * FROM {table}"
    dataframes[table] = pd.read_sql_query(query, sales_connection)

product_df = dataframes['product']
product_type_df = dataframes['product_type']
product_line_df = dataframes['product_line']
sales_staff_df = dataframes['sales_staff']
sales_branch_df = dataframes['sales_branch']
retailer_site_df = dataframes['retailer_site']
country_df = dataframes['country']
order_header_df = dataframes['order_header']
order_details_df = dataframes['order_details']
returned_item_df = dataframes['returned_item']
return_reason_df = dataframes['return_reason']

print(product_df)
print(product_type_df)

    PRODUCT_NUMBER INTRODUCTION_DATE PRODUCT_TYPE_CODE PRODUCTION_COST MARGIN  \
0                1         15-2-2011                 1               4    .33   
1               10         15-2-2011                 1              10     .4   
2              100         15-2-2011                17               3     .5   
3              101        15-12-2019                18          305.54    .43   
4              102        10-12-2019                18          380.95    .51   
..             ...               ...               ...             ...    ...   
110             95         15-2-2011                16               3     .5   
111             96         15-2-2011                17           16.43    .28   
112             97          5-3-2013                17              25    .28   
113             98         15-2-2011                17               3     .5   
114             99         15-2-2011                17               2     .6   

       PRODUCT_IMAGE LANGUA

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [22]:
product_df['PRODUCTION_COST'] = pd.to_numeric(product_df['PRODUCTION_COST'], errors='coerce')
producten_filtered = product_df.loc[(product_df['PRODUCTION_COST'] > 50) & (product_df['PRODUCTION_COST'] < 100), ['PRODUCT_NAME', 'PRODUCTION_COST']]

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [23]:
product_df['MARGIN'] = pd.to_numeric(product_df['MARGIN'], errors='coerce')
result = product_df[['PRODUCT_NAME', 'MARGIN']][(product_df['MARGIN'] < 0.20) | (product_df['MARGIN'] > 0.60)]

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [24]:
countries_francs = country_df[['COUNTRY', 'CURRENCY_NAME']][country_df['CURRENCY_NAME'] == 'francs']
# countries_francs

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [25]:
product_df['MARGIN'] = pd.to_numeric(product_df['MARGIN'], errors='coerce')
products_high_margin = product_df[['INTRODUCTION_DATE', 'MARGIN']][product_df['MARGIN'] > 0.50]
products_high_margin_unique_dates = products_high_margin.drop_duplicates(subset='INTRODUCTION_DATE')
# products_high_margin_unique_dates

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [26]:
verkoopafdelingen = sales_branch_df[['ADDRESS1', 'CITY']][(sales_branch_df['ADDRESS2'].notna()) & (sales_branch_df['REGION'].notna())]
# verkoopafdelingen

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [27]:
dollarNewDollar = country_df[['COUNTRY']][(country_df['CURRENCY_NAME'] == 'dollars') | (country_df['CURRENCY_NAME'] == 'new dollar')]
# dollarNewDollar

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [28]:
clients = retailer_site_df[['ADDRESS1', 'ADDRESS2', 'CITY']][(retailer_site_df['POSTAL_ZONE'].str.startswith('D')) & (retailer_site_df['ADDRESS2'].notnull())]
# clients

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [29]:
returned_item_df['RETURN_QUANTITY'] = pd.to_numeric(returned_item_df['RETURN_QUANTITY'])
total_returned_items = [returned_item_df['RETURN_QUANTITY'].sum()]
# total_returned_items

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [30]:
region_amount = sales_branch_df['REGION'].count()
# region_amount

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [31]:
laagste = product_df['MARGIN'].min()
hoogste = product_df['MARGIN'].max()
gemiddelde = product_df['MARGIN'].mean()
resultaten = pd.DataFrame({
    'Laagste': [laagste],
    'Hoogste': [hoogste],
    'gemiddelde': [gemiddelde]
})
# resultaten

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [32]:
nonAvailableAdress = retailer_site_df['ADDRESS2'].isnull().sum()
# nonAvailableAdress

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [33]:
order_details_df['UNIT_COST'] = pd.to_numeric(order_details_df['UNIT_COST'])
averageCostPriceDiscount = order_details_df[['UNIT_COST']][(order_details_df['UNIT_SALE_PRICE'] < order_details_df['UNIT_PRICE'])].mean()
# averageCostPriceDiscount

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [34]:
medewerkersPerFunctie = sales_staff_df.groupby('POSITION_EN')['SALES_STAFF_CODE'].count()
# medewerkersPerFunctie

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [35]:
medewerkers_per_telefoonnummer = sales_staff_df.groupby('WORK_PHONE')['SALES_STAFF_CODE'].count().reset_index()
telefoonnummers_meer_dan_4_medewerkers = medewerkers_per_telefoonnummer[medewerkers_per_telefoonnummer['SALES_STAFF_CODE'] > 4]
# telefoonnummers_meer_dan_4_medewerkers

## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [36]:
klantenUitNederland = retailer_site_df[['ADDRESS1', 'CITY']][retailer_site_df['COUNTRY_CODE'] == '7']
# klantenUitNederland

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [37]:
eyewareProducts = product_df['PRODUCT_NAME'][product_df['PRODUCT_TYPE_CODE'] == '11'] 
# eyewareProducts

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [38]:
merged = pd.merge(sales_staff_df, sales_branch_df, on='SALES_BRANCH_CODE', how='left')
branch_managers = merged[merged['POSITION_EN'] == 'Branch Manager']
unique_addresses = merged['ADDRESS1'].unique()
result_df = branch_managers[branch_managers['ADDRESS1'].isin(unique_addresses)][['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']]
# result_df

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [39]:
merged_df = pd.merge(sales_staff_df, sales_branch_df, on='SALES_BRANCH_CODE', how='left')
merged_order_df = pd.merge(merged_df, order_header_df, on=['SALES_STAFF_CODE', 'SALES_STAFF_CODE'], how='inner')
manager_positions = merged_order_df[merged_order_df['POSITION_EN'].str.contains('Manager')]
result_df = manager_positions[['POSITION_EN', 'ORDER_DATE']]
# result_df

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [40]:
order_details_df['QUANTITY'] = pd.to_numeric(order_details_df['QUANTITY'], errors='coerce')
high_quantity_orders = order_details_df[order_details_df['QUANTITY'] > 750]
merged_product_info = pd.merge(high_quantity_orders, product_df, on='PRODUCT_NUMBER', how='inner')
result_df = merged_product_info[['PRODUCT_NAME', 'PRODUCT_TYPE_CODE']]
result_df = result_df.drop_duplicates()
# result_df

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [41]:
order_details_df['UNIT_PRICE'] = pd.to_numeric(order_details_df['UNIT_PRICE'], errors='coerce')
order_details_df['UNIT_SALE_PRICE'] = pd.to_numeric(order_details_df['UNIT_SALE_PRICE'], errors='coerce')
merged_df = pd.merge(product_df, order_details_df, on='PRODUCT_NUMBER', how='inner')
merged_df['DISCOUNT_PERCENTAGE'] = ((merged_df['UNIT_PRICE'] - merged_df['UNIT_SALE_PRICE']) / merged_df['UNIT_PRICE']) * 100
high_discount_products = merged_df[merged_df['DISCOUNT_PERCENTAGE'] > 40]
result_df = high_discount_products[['PRODUCT_NAME']].drop_duplicates()
# result_df

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [42]:
merged_returns_reason_df = pd.merge(returned_item_df, return_reason_df, on='RETURN_REASON_CODE', how='left')
merged_returns_df = pd.merge(order_details_df, merged_returns_reason_df, on='ORDER_DETAIL_CODE', how='inner')
merged_returns_df['QUANTITY'] = pd.to_numeric(merged_returns_df['QUANTITY'], errors='coerce')
merged_returns_df['RETURN_QUANTITY'] = pd.to_numeric(merged_returns_df['RETURN_QUANTITY'], errors='coerce')
merged_returns_df['RETURN_PERCENTAGE'] = (merged_returns_df['RETURN_QUANTITY'] / merged_returns_df['QUANTITY']) * 100
high_return_products = merged_returns_df[merged_returns_df['RETURN_PERCENTAGE'] > 90]
result_df = high_return_products[['RETURN_DESCRIPTION_EN']].drop_duplicates()
# result_df

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [43]:
merged_products_orders_df = pd.merge(product_df, order_details_df, on='PRODUCT_NUMBER', how='left')
product_type_counts = merged_products_orders_df.groupby('PRODUCT_TYPE_CODE')['PRODUCT_NUMBER'].nunique().reset_index()
product_type_counts.columns = ['PRODUCT_TYPE_CODE', 'NUMBER_OF_PRODUCTS']
# product_type_counts

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [44]:
merged_retailer_sites_df = pd.merge(retailer_site_df, country_df, left_on='COUNTRY_CODE', right_on='COUNTRY_CODE', how='inner')
retailer_sites_by_country = merged_retailer_sites_df.groupby(['COUNTRY'])['RETAILER_SITE_CODE'].nunique().reset_index()
# retailer_sites_by_country

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [45]:
cooking_gear_products = product_df[product_df['PRODUCT_TYPE_CODE'] == '1']
merged_df = pd.merge(cooking_gear_products, order_details_df, on='PRODUCT_NUMBER', how='inner')
merged_df = pd.merge(merged_df, product_type_df, on='PRODUCT_TYPE_CODE', how='inner')
result_df = merged_df.groupby('PRODUCT_NAME').agg({
    'QUANTITY': 'sum',
    'UNIT_SALE_PRICE': 'mean',
    'PRODUCT_TYPE_EN': 'first'
}).reset_index()
result_df = result_df.sort_values(by='QUANTITY', ascending=False)
# result_df

Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

In [46]:
merged_df = pd.merge(sales_staff_df, sales_branch_df, on='SALES_BRANCH_CODE', how='left')
sales_staff_amsterdam_df = merged_df[merged_df['CITY'] == 'Amsterdam']
result_df = sales_staff_amsterdam_df[['FIRST_NAME', 'LAST_NAME', 'EXTENSION']]
# result_df

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [47]:
merged_sales_branch = pd.merge(sales_branch_df, country_df, on='COUNTRY_CODE', how='left')
grouped_data = merged_sales_branch.groupby(['COUNTRY', 'CITY']).size().reset_index(name='Klanten')
result_df = grouped_data[['COUNTRY', 'CITY', 'Klanten']]
result_df.columns = ['Land', 'Verkoper', 'Klanten']
# result_df

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [48]:
# Find all sales staff codes in order_header_df
existing_codes = order_header_df['SALES_STAFF_CODE']

# Filter sales_staff_df to exclude codes in existing_codes
staff_without_sales = sales_staff_df[~sales_staff_df['SALES_STAFF_CODE'].isin(existing_codes)]
selected_columns = ['FIRST_NAME', 'LAST_NAME']
filtered_staff = staff_without_sales[selected_columns]

# Print the DataFrame
#filtered_staff

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [49]:
numericMargin = pd.to_numeric(product_df['MARGIN'])
total_margin = numericMargin.sum()
# shape wordt gebruikt om het aantal rijen en kolommen te bepalen door[0] bepaal je aantal rijen en met [1] aantal kolommen
total_Products = product_df['PRODUCT_NUMBER'].count()
avg_margin_product = total_margin/total_Products
low_margin_products = product_df['MARGIN'] < avg_margin_product

total_low_margin = low_margin_products.sum()
data = {
    "Average Margin": [avg_margin_product],
    "Products Below Avg": [low_margin_products.sum()]
}

overview_df = pd.DataFrame(data)

# Print the DataFrame
print(overview_df)

   Average Margin  Products Below Avg
0        0.396609                  59


Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [50]:
products_over500 = product_df["PRODUCTION_COST"] * (1 +product_df["MARGIN"]) > 500
over500_products_df = product_df[products_over500]
over500_product_names = over500_products_df["PRODUCT_NAME"]
over500_product_names #klopt niet met de 13 rijen die verwacht worden en ik moet ook nog de check uitvoeren of ze nog nooit zijn teruggebracht

4              Hailstorm Titanium Irons
6         Lady Hailstorm Titanium Irons
7          Hailstorm Titanium Woods Set
8             Hailstorm Steel Woods Set
9     Lady Hailstorm Titanium Woods Set
10       Lady Hailstorm Steel Woods Set
19                            Star Dome
21                         Star Gazer 3
22                         Star Gazer 6
55                       Husky Rope 200
Name: PRODUCT_NAME, dtype: object

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [51]:
naam_functie = sales_staff_df['LAST_NAME'], sales_staff_df['POSITION_EN']
naam_functie

(0      Savolainen
 1             Kim
 2            Choi
 3             Kao
 4            Fang
           ...    
 97       Iglesias
 98          Lopes
 99         Barros
 100     Guimarães
 101      Lindholm
 Name: LAST_NAME, Length: 102, dtype: object,
 0      Level 2 Sales Representative
 1      Level 2 Sales Representative
 2      Level 3 Sales Representative
 3      Level 2 Sales Representative
 4      Level 3 Sales Representative
                    ...             
 97     Level 2 Sales Representative
 98                   Branch Manager
 99     Level 3 Sales Representative
 100    Level 2 Sales Representative
 101    Level 3 Sales Representative
 Name: POSITION_EN, Length: 102, dtype: object)

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [52]:
from datetime import date
date.today().year

2024

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [53]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 